## Imports

In [6]:
#Delete me
# Settings for notebook visualization
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%matplotlib inline
from IPython.core.display import HTML
HTML("""<style>.output_png img {display: block;margin-left: auto;margin-right: auto;text-align: center;vertical-align: middle;} </style>""")

In [7]:
# Necessary imports
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import quantstats as qs
import statistics as st
from datetime import datetime, timedelta
from matplotlib.colors import DivergingNorm
from scipy.signal import convolve2d
from IPython.core.display import HTML
HTML("""<style>.output_png img {display: block;margin-left: auto;margin-right: auto;} </style>""")

In [8]:
# Other settings
qs.extend_pandas()

# Settings for plot visualization
plt.style.use('seaborn-darkgrid')

#plt.rcParams.keys()
plt.rcParams['figure.dpi'] = 200
plt.rcParams["figure.figsize"] = (12,3.5) #(12,5)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linewidth'] = 0.4
#plt.rcParams['xtick.label.allignment'] = 'center'

plt.rcParams['xtick.bottom'] = plt.rcParams['ytick.labelright'] = True
plt.rcParams['ytick.left'] = plt.rcParams['ytick.right'] = True

plt.rcParams['lines.linewidth'] = 1.2
#plt.rcParams['lines.markersize'] = 0.5
plt.rcParams['patch.edgecolor'] = 'k' # Legend border 
plt.rcParams['legend.facecolor'] = 'w'
plt.rcParams["legend.frameon"] = True

np.set_printoptions(edgeitems=40, linewidth=1000)

pd.set_option("display.precision", 6)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#print("Notebook parameters set correctly")

## Data

In [9]:
ini_equity_default = 100
commision_default = 0.000111538462 # 2/130000 + 12.5/130000
#commision_default = 0.0005 # Slightly Bbgger commision, for better visualization
# 0.01 = 1% of the cummulative return (equity)

In [63]:
#https://finance.yahoo.com/quote/%5EIRX?p=^IRX&.tsrc=fin-srch
# 13-week treasury bills
# All treasury bills from yahoo: https://finance.yahoo.com/bonds?.tsrc=fin-srch
# rf_13w = yf.download("^IRX", auto_adjust=True, start="1928-01-01")
# rf_10y = yf.download("^TNX", auto_adjust=True, start="1928-01-01")
# rf = pd.concat([rf_13w['Close'], rf_10y['Close']], axis=1)
# rf.columns = ['13W', '10Y']
# rf.tail()
#rf.plot()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,13W,10Y
Date,,
2020-11-04,0.083,0.768
2020-11-05,0.085,0.776
2020-11-06,0.083,0.820
2020-11-09,0.088,0.958
2020-11-10,0.090,0.972


In [10]:
# Load DF with SP500 data
def get_sp500_data(start_date="1928-01-01", from_local_file=True, save_to_file=False):
    if from_local_file == True:
        data = pd.read_pickle('data/SP500_hist_data.pkl')
    else:
        # Download data from yfinance
        data = yf.download("^GSPC", auto_adjust=True, start=start_date)
        if save_to_file == True:
            data.to_pickle("data/SP500_hist_data.pkl")
    return data

In [89]:
full_df = get_sp500_data()
full_df['Market_daily_ret'] = full_df['Close'].pct_change().fillna((full_df['Close']-full_df['Open'])/full_df['Open'])
full_df = full_df[['Close', 'Open', 'Market_daily_ret']]

## Strategy functions

In [75]:
# Define if the strategy position (1=fast_ma higher than slow_ma, -1=short)
def get_strategy_position_buy_and_hold(df):
    position = pd.Series(1.0, index=df.index, dtype='float64')
    
    return position

In [12]:
# Define if the strategy position (1=fast_ma higher than slow_ma, -1=short)
def get_strategy_position_ma(df, fast_ma=0, slow_ma=0):
    df['fast_ma'] = full_df['Close'].rolling(window=fast_ma).mean()
    df['slow_ma'] = full_df['Close'].rolling(window=slow_ma).mean()
    df['fast-slow'] = df['fast_ma'].sub(df['slow_ma'])
    
    conditions = [df['fast-slow'] > 0, df['fast-slow'] < 0, df['fast-slow'] == 0]
    signals = [1.0, 0.0, 1.0]

    position = pd.Series(np.select(conditions, signals), index=df.index, dtype='float64')
        
    return position

## Backtest functions

In [13]:
def get_cum_ret_from_daily_pct_ret(daily_returns, costs=0.0, ini_equity=ini_equity_default):    
    if (isinstance(costs, pd.Series) == False):
        costs = pd.Series(0, index=daily_returns.index)
    
    # Calculate cummulative returns as: cum_returns = ini_equity * CUMPROD ((1+daily_ret) * (1-costs))
    cum_returns = daily_returns.add(1).mul(1-costs).cumprod().mul(ini_equity)
    
    return cum_returns

In [15]:
def backtest_print_plot(df, fast_ma=0, slow_ma=0, previous_position=0, ini_equity=ini_equity_default, commision=commision_default, figsize=(12,3.5), legend=False): #(12,5)
    df, _, ret_strat, ir_strat= backtest_strat_ma(df, fast_ma=fast_ma, slow_ma=slow_ma, ini_equity=ini_equity, previous_position=previous_position, commision=commision)
    print_backtest_stats_ma(df, fast_ma=fast_ma, slow_ma=slow_ma, ret_strat=ret_strat, ir_strat=ir_strat)
    show_plot(df, figsize=figsize, legend=legend)
    return df

In [16]:
"""
backtest_strat_ma does the backtest of an MA crossover strategy. It adds the following columns to the received dataframe:
"""
def backtest_strat_ma(df, fast_ma=0, slow_ma=0, ini_equity=ini_equity_default, previous_position=0, commision=commision_default):
    if ((fast_ma == slow_ma == 0) | (fast_ma >= slow_ma)): # Long only
        first_position = 1
        df['Strat_position'] = 1
        df['Long_only'] = 1
    else:
        loc = full_df.index.get_loc(df.index[0])
        first_position = get_strategy_position_ma(full_df.iloc[loc - 1].to_frame().T, fast_ma, slow_ma)[0]
        df['Strat_position'] = get_strategy_position_ma(df.copy(), fast_ma, slow_ma)
        df['Long_only'] = 0
    
    df = get_returns_from_strat_position(df, ini_equity=ini_equity, previous_position=previous_position, first_position=first_position, commision=commision)
#    df.columns= ['Close', 'Market_daily_ret', 'Strat_daily_ret', 'Market_cum_ret', 'Strat_cum_ret', 'Costs', 'Strat_position', 'Long_only']
    
    last_position = df.loc[df.index[-2], 'Strat_position']
    df.loc[df.index[-1], 'Strat_position'] = np.nan
    
    ret_strat = (df.loc[df.index[-1], 'Strat_cum_ret'] / ini_equity) * 100
    ir_strat = calculate_information_ratio(df['Strat_cum_ret'])
    
    return df, last_position, ret_strat, ir_strat

'\nbacktest_strat_ma does the backtest of an MA crossover strategy. It adds the following columns to the received dataframe:\n'

In [91]:
"""
Given 'Market_daily_ret' and 'Strat_position', adds to df: 'Strat_daily_ret', 'Market_cum_ret', 'Strat_cum_ret', 'Costs'
Formula:
   - Strat_cum_ret = [(1+Market_daily_ret*Strat_position.shift()) * (1-commission_paid_pct)].cumprod() * ini_equity
"""
def get_returns_from_strat_position(df, ini_equity=ini_equity_default, previous_position=0, first_position=0 , commision=commision_default):
    # Previous_position only affect costs. represents the position where we come from. It will be used to determine if we change a position right before our backtest
    # First position affects cummulative returns. Position for the first day will be the same as for the second day
    
    # commission_paid_pct will place a commision on the days that we have a change in Strat_position
    df['commission_paid_pct'] = df['Strat_position'].sub(df['Strat_position'].shift(fill_value=previous_position)) \
                                .abs() \
                                .mul(commision)
    
    # Strat_cum_ret = [(1+Market_daily_ret*Strat_position) * (1-commission_paid_pct)].cumprod() * ini_equity
    df['Strat_cum_ret'] = df['Market_daily_ret'].mul(df['Strat_position'].shift(fill_value=first_position)).add(1) \
                            .mul(1-df['commission_paid_pct']) \
                            .cumprod() \
                            .mul(ini_equity)
#    df.loc[df.index[0], 'Strat_cum_ret'] = ini_equity * (1 + df['Market_daily_ret'][0]*df['Strat_position'][0]) * (1-df['commission_paid_pct'][0])
    
    # Costs (in USD) = Strat_cum_ret.shift() * [[Market_Daily_ret*Strat_position.shift()] + 1] * (commission_in_pct)
    df['Costs'] = df['Strat_cum_ret'].shift(fill_value=ini_equity) \
            .mul(df['Market_daily_ret'].mul(df['Strat_position'].shift(fill_value=previous_position)).add(1)) \
            .mul(df['commission_paid_pct'])

    df['Strat_daily_ret'] = df['Strat_cum_ret'].pct_change(fill_value=ini_equity)
    
    # Market returns
    df['Market_cum_ret'] = get_cum_ret_from_daily_pct_ret(df['Market_daily_ret'], ini_equity=ini_equity)
    
    cols = ['Close', 'Market_daily_ret', 'Strat_daily_ret', 'Strat_position', 'Costs', 'Long_only', 'Market_cum_ret', 'Strat_cum_ret']
    df = df.loc[:, cols]

    df.loc[df.index[-1], 'Strat_position'] = np.nan # We leave the last position (position for the following day) as NaN    
    
    return df

"\nGiven 'Market_daily_ret' and 'Strat_position', adds to df: 'Strat_daily_ret', 'Market_cum_ret', 'Strat_cum_ret', 'Costs'\nFormula:\n   - Strat_cum_ret = [(1+Market_daily_ret*Strat_position.shift()) * (1-commission_paid_pct)].cumprod() * ini_equity\n"

## Walk-forward functions

In [90]:
"""
Builds the cummulative return of the strategy and market between two dates.
Receives a df with the performance of several OOS periods joined in 'Strat_daily_ret'
"""
def prepare_oos_df(df, ini_equity=ini_equity_default, commision=commision_default):
    cols = ['Close', 'Market_daily_ret', 'Strat_daily_ret', 'Strat_position', 'Long_only', 'Costs']
    results_df = df[cols].copy()

    first_pos = results_df.loc[results_df.index[0], 'Strat_position'] 
    last_pos = results_df.loc[results_df.index[-2], 'Strat_position']
    
    # Replace NaN values in 'Strat_position' (last one from each OOS window) by the next 'Strat_postion' value
    results_df['Strat_position'].fillna(results_df['Strat_position'].shift(-1, fill_value=last_pos), inplace=True)
    
    results_df = get_returns_from_strat_position(results_df, ini_equity=ini_equity, commision=commision, previous_position=0, first_position=first_pos)
    
    return results_df

"\nBuilds the cummulative return of the strategy and market between two dates.\nReceives a df with the performance of several OOS periods joined in 'Strat_daily_ret'\n"

In [19]:
def print_periods(IS_start_years, IS_end_years, OOS_start_years, OOS_end_years):
    print("\tIn SAMPLE\t\tOOS")
    for iss, ie, oi, oe in zip(IS_start_years, IS_end_years, OOS_start_years, OOS_end_years):
        print("{:%Y-%m-%d} : {:%Y-%m-%d} \t {:%Y-%m-%d} : {:%Y-%m-%d}".format(iss, ie, oi, oe))
    print("Number of periods: {} : {}     {} : {}".format(len(IS_start_years), len(IS_end_years), len(OOS_start_years), len(OOS_end_years)))

In [20]:
"""
Checks how many nan are around each element of a matrix. Useful to see the real numbers of neighbors of each element in the matrix.
Receives a matrix of type numpy.ndarray
"""
def check_nan_around_matrix(matrix):
    n_rows = matrix.shape[0]
    n_cols = matrix.shape[1]

    num_nan_around = np.full((n_rows, n_cols), 0)

    for i in range(n_rows):
        for j in range(n_cols):
            counter = 0
            for ii in range(i-1, i+2):
                if (ii >= 0) and (ii < n_rows):
                    for jj in range(j-1, j+2):
                        if (jj >= 0) and (jj < n_cols):
                            if np.isnan(matrix[ii, jj]) == True:
                                counter += 1
            num_nan_around[i,j] = counter

    return num_nan_around

'\nChecks how many nan are around each element of a matrix. Useful to see the real numbers of neighbors of each element in the matrix.\nReceives a matrix of type numpy.ndarray\n'

In [21]:
"""
Checks how many neighbors has each element of a matrix.
"""
def get_num_neighbors(param_1_list, param_2_list):
    n_rows = len(param_1_list)
    n_cols = len(param_1_list)
    
    n_neighbors = np.full((n_rows, n_cols), 8.0) # Default number of neighbors of each cell
    n_neighbors[[0,n_cols-1], :] = n_neighbors[:, [0,n_cols-1]] = 5 # Edges
    n_neighbors[[0,n_cols-1], [0,n_cols-1]] = n_neighbors[[n_rows-1,0], [0,n_cols-1]] = 3 # Corners

    for i in range(n_rows):
        for j in range(n_cols):
            if param_1_list[i] >= param_2_list[j]:
                n_neighbors[i,j] = np.nan

    num_notnan_neighbors = n_neighbors - check_nan_around_matrix(n_neighbors)
    
    return num_notnan_neighbors

'\nChecks how many neighbors has each element of a matrix.\n'

In [22]:
"""
Receives SR of market and each tested parameter combination to return the best one. 
Best one = max((SR of each element)*50% + (average SR of its neightbors)*50%)
"""
def get_best_combination(market_ir, results_ir, num_neighbors_matrix, allow_long_only=True):
    n = len(results_ir)

    sum_ir_neighbors = convolve2d(results_ir, np.ones((3,3)),'same') - results_ir
        
    results_ir_neighbors = np.divide(sum_ir_neighbors, num_neighbors_matrix)
    
    # (Individual SR + neighbors SR) / 2
    results_ir_combined = np.divide(np.add(results_ir, results_ir_neighbors), 2)
    
    results_ir_combined = np.nan_to_num(results_ir_combined, nan=market_ir)
    
    # Get index from best SR
    fast_index, slow_index = np.unravel_index(np.argmax(results_ir_combined, axis=None), results_ir_combined.shape)

    #print("Individual: {}-{}".format(fast_ma[fast_index_ind], slow_ma[slow_index_ind]))
    #print("With NN: {}-{}".format(fast_ma[fast_index], slow_ma[slow_index]))
    #print("Best: {}-{}".format(fast_ma[fast_index], slow_ma[slow_index]))

    return fast_index, slow_index, results_ir_combined

'\nReceives SR of market and each tested parameter combination to return the best one. \nBest one = max((SR of each element)*50% + (average SR of its neightbors)*50%)\n'

In [23]:
"""
Runs a backtest with all possible combinations and returns 2 matrices, one with pnl results, and one with SR
In MA crossover strategy, param_1 refers to fast_ma. param_2 refers to slow_ma

"""
def run_all_combinations(df, param_1_list, param_2_list, last_position):
    results_pnl = np.zeros((len(param_1_list),len(param_2_list)))
    results_ir = np.zeros((len(param_1_list),len(param_2_list)))

    _, _, SP_pnl, SP_ir = backtest_strat_ma(df, fast_ma=0, slow_ma=0)

    for i, param_1 in enumerate(param_1_list):
        for j, param_2 in enumerate(param_2_list):
            if param_1 < param_2:
                _, _, pnl, ir = backtest_strat_ma(df, fast_ma=param_1, slow_ma=param_2, previous_position=last_position)
                results_pnl[i,j] = pnl
                results_ir[i,j] = ir

    return results_pnl, results_ir, SP_ir

'\nRuns a backtest with all possible combinations and returns 2 matrices, one with pnl results, and one with SR\n'

## Plots

In [93]:
def show_plot(df, benchmark=True, position=True, fast_ma=1, slow_ma=1, figsize=(12,3.6), legend=False, with_signals=False): #(12,5)
    df_plot = df.copy()
    first_day = df_plot.index[0]
    last_day = df_plot.index[-1]
    
    fmt = '%Y-%m-%d' if first_day.year == last_day.year else '%Y'
        
    columns_colors_ax1 = [('Strat_cum_ret', 'b')]
    
    title = 'MA crossover strategy ({} : {})'.format(first_day.strftime(fmt), last_day.strftime(fmt))

    if (benchmark == True):
        title = 'SP500 vs ' + title
        columns_colors_ax1.append(('Market_cum_ret', 'k'))

    # PLOT
    if position == True:
        import matplotlib.dates as mdates
        
        fig, ax1 = plt.subplots(figsize=figsize)
        ax2 = ax1.twinx()

        [ax1.plot(df_plot.index, df_plot[column], label=column, color=color) for (column, color) in columns_colors_ax1[::-1]]
        
        columns_label_colors_ax2 = [('Strat_position', 'Strategy position', 'r')]
        columns_label_colors_ax2 = [('Long_only', 'Only long position allowed', 'tab:brown'), ('Strat_position', 'Strategy position', 'g')]
        df_plot['Long_only'] = df_plot['Long_only'] - 0.03
        
        [ax2.scatter(df_plot.index, df_plot[column], marker='s', s=1, label=lab, color=color) for (column, lab, color) in columns_label_colors_ax2]            
        ax2.set(ylabel='Strategy Position', yticks=np.arange(-1.0,1.01,1))
        ax2.set_ylim([-1.1,1.1])
        ax2.tick_params(axis='y', direction='out', length=5)
        
        ax2.grid(False)
        
        handles, labels = ax2.get_legend_handles_labels()
        if legend == True:
            ax2.legend(handles[::-1], labels[::-1], loc='upper left', bbox_to_anchor=(0.55, -0.08), borderpad=0.5, markerscale=3) #borderpad=1
        
    else:
        [ax1.plot(df_plot.index, df_plot[column], label=column, color=color) for (column, color) in columns_colors_ax1[::-1]]

    from matplotlib import ticker
    
    # X-AXES
    # Write a max of 20 major locators in x axis
    n_years = last_day.year - first_day.year
    freq = 1 if (n_years <= 20) else 5
    years = mdates.YearLocator(freq)       
    yearFmt = mdates.DateFormatter('%Y')
    # Add the locators to the axis
    ax1.xaxis.set_major_locator(years)
    ax1.xaxis.set_major_formatter(yearFmt) # Add tick with every freq years
    ax1.xaxis.set_minor_locator(mdates.YearLocator(1)) # every year
    ax1.tick_params(axis='x', direction='out', which='major', length=4)
    ax1.tick_params(axis='x', direction='out', which='minor', length=2)  
    
    loc = full_df.index.get_loc(first_day)
    first_tick = full_df.index.values[loc-1]
    loc = full_df.index.get_loc(last_day)
    loc = loc if (loc == full_df.index.size - 1) else loc + 1 # Necessary to add one day checking for index size
    last_tick = full_df.index.values[loc]
    ax1.set_xlim([first_tick, last_tick])
    
    # Y-AXES
    ax1.set(ylabel='Value (USD)', title=title)
    ax1.tick_params(axis='y', direction='out', length=4)
    #ax1.yaxis.set_minor_locator(ticker.MultipleLocator(250))
    #ax1.tick_params(axis='y', direction='out', which='minor', length=2)
    
    # LEGEND
    if legend == True:
        # Put a legend below current axis, reversing the printing order and incresing linewidth
        handles, labels = ax1.get_legend_handles_labels()
        repl = {'Strat_cum_ret':'Strategy', 'Market_cum_ret':'Benchmark', 'Strat_position':'Strategy position'}
        repl = {'Strat_cum_ret':'Strategy', 'Market_cum_ret':'Benchmark', 'Long_only':'Only long position allowed', 'Strat_position':'Strategy position'}
        labels = [repl.get(n, n) for n in labels]
        leg1 = ax1.legend(handles[::-1], labels[::-1], loc='upper right', bbox_to_anchor=(0.45, -0.08), borderpad=0.5)
        [legobj.set_linewidth(3) for legobj in leg1.legendHandles]
    
    if with_signals == True:
        buy_signal = df_plot['Strat_position'] > df_plot['Strat_position'].shift()
        buy_marker = df_plot['Close']
        buy_marker = buy_marker[buy_signal]
        buy_dates = df_plot.index[buy_signal]
        sell_signal = df_plot['Strat_position'] < df_plot['Strat_position'].shift()
        sell_marker = df_plot['Close']
        sell_marker = sell_marker[sell_signal]
        sell_dates = df_plot.index[sell_signal]
        ax1.scatter(buy_dates, buy_marker, marker='^', color='green', label='Buy');
        ax1.scatter(sell_dates, sell_marker, marker='>', color='red', label='Exit');

    # 
    ax1.set_zorder(1)
    ax1.patch.set_visible(False)
    ax2.set_zorder(0)
    ax2.patch.set_visible(True)
    
    #plt.show()

In [30]:
def plot_sp500_with_ma_signals(df, fast_ma, slow_ma):
    first_day = df.index[0]
    last_day = df.index[-1]
    
    fmt = '%Y-%m-%d' if first_day.year == last_day.year else '%Y'

    df['fast_ma'] = full_df['Close'].rolling(window=fast_ma).mean()[first_day:last_day]
    df['slow_ma'] = full_df['Close'].rolling(window=slow_ma).mean()[first_day:last_day]
    df['diff'] = df['fast_ma'].sub(df['slow_ma'])
    df['long_signal'] = (df['diff'] > 0) & (df['diff'].shift(1).fillna(-1) <= 0)
    df['exit_signal'] = (df['diff'] <= 0) & (df['diff'].shift(1) > 0)

    buy_marker = df['fast_ma'] * df['long_signal'] - (df['fast_ma'].max()*.01)
    buy_marker = buy_marker[df['long_signal']]
    buy_dates = df.index[df['long_signal']]
    exit_marker = df['fast_ma'] * df['exit_signal'] - (df['fast_ma'].max()*.01)
    exit_marker = exit_marker[df['exit_signal']]
    exit_dates = df.index[df['exit_signal']]

    title = "SP500 with MAs of {}-{} ({} : {})".format(fast_ma, slow_ma, first_day.strftime(fmt), last_day.strftime(fmt))
    
    fig = plt.figure()
    ax = df[['Close', 'fast_ma', 'slow_ma']].plot(title=title,
                                             figsize=(14,6), 
                                             color=('k', 'r', 'y'))
    
    ax.scatter(buy_dates, buy_marker, marker='^', color='green', label='Buy');
    ax.scatter(exit_dates, exit_marker, marker='>', color='blue', label='Exit');

    ax.legend(["SP500", 'MA '+ str(fast_ma), 'MA '+ str(slow_ma)]);
    ax.tick_params(axis='x', direction='out', length=3, labelrotation=0);
    ax.tick_params(axis='y', direction='out', length=3);
    plt.xticks(horizontalalignment="center");

In [31]:
def show_oos_plot(results_df, legend=False, with_signals=False):
#    show_plot(results_df, start=str(results_df.index[0]), end=str(results_df.index[-1]))
    show_plot(results_df, legend=legend, with_signals=with_signals)

In [80]:
# Plots a heatmap with data from a matrix. 
def show_heatmap(data, market_ir, plot_title, x_title, x_values, y_title, y_values):
    # Flip matrix vertically for better visualization
    data = np.flip(data, axis=0)
    #data = np.nan_to_num(data, nan=market_ir)
    fig, ax = plt.subplots(figsize=(3, 3)) #11,9

    rdgn = sns.diverging_palette(h_neg=10, h_pos=130, as_cmap=True, s=80, l=50)
#    divnorm = DivergingNorm(vmin=data.min(), vcenter=0, vmax=data.max())
#    sns.heatmap(data, cmap=rdgn, norm=divnorm, annot=True, fmt ='.2', 
    fig = sns.heatmap(data, cmap=rdgn, annot=True, fmt =".3f", annot_kws={"fontsize":5}, 
                vmin=-1.0, center=0, vmax=1.0,
                linecolor='black', cbar=True, ax=ax,
                xticklabels=x_values, yticklabels=np.flip(y_values))
    cbar = fig.collections[0].colorbar
    cbar.ax.tick_params(labelsize=5);
    ax.yaxis.tick_left();
    plt.xlabel(x_title, fontsize=6);
    plt.ylabel(y_title, fontsize=6);
    plt.title(plot_title, fontsize=7);
    plt.xticks(fontsize=6, rotation=0);
    plt.yticks(fontsize=6, rotation=0);

In [86]:
# Plots a heatmap with data from a matrix. 
def show_both_heatmaps(data_individual, data_robust, market_ir, plot_title, x_title, x_values, y_title, y_values):
    # Flip matrix vertically for better visualization
    data_individual = np.flip(data_individual, axis=0)
    data_robust = np.flip(data_robust, axis=0)

    #data = np.nan_to_num(data, nan=market_ir)
    fig, ax = plt.subplots(ncols=2, figsize=(10, 4)) #11,9
    fig.subplots_adjust(wspace=0.2)
    fig.suptitle("Information Ratio In Sample (" + plot_title +")", fontsize=9)
    ax[0].set_title("Individual", fontsize=7)
    ax[1].set_title("Robust IR = (Individual + neighbors.mean()) / 2", fontsize=7)
    
    
    rdgn = sns.diverging_palette(h_neg=10, h_pos=130, as_cmap=True, s=80, l=50)
#    divnorm = DivergingNorm(vmin=data.min(), vcenter=0, vmax=data.max())
#    sns.heatmap(data, cmap=rdgn, norm=divnorm, annot=True, fmt ='.2', 
    sns.heatmap(data_individual, cmap=rdgn, annot=True, fmt =".3f", annot_kws={"fontsize":5}, 
                vmin=-1.0, center=0, vmax=1.0,
                linecolor='black', cbar=False, ax=ax[0],
                xticklabels=x_values, yticklabels=np.flip(y_values))
    fig.colorbar(ax[0].collections[0], ax=ax[0], location="left", use_gridspec=False, pad=0.2)

    sns.heatmap(data_robust, cmap=rdgn, annot=True, fmt =".3f", annot_kws={"fontsize":5}, 
                vmin=-1.0, center=0, vmax=1.0,
                linecolor='black', cbar=False, ax=ax[1],
                xticklabels=x_values, yticklabels=np.flip(y_values))
    fig.colorbar(ax[1].collections[0], ax=ax[1], location="right", use_gridspec=False, pad=0.2)
    ax[1].yaxis.set_label_position("right")
    #ax[1].yaxis.tick_right()
    
    for a in ax:
        a.collections[0].colorbar.ax.tick_params(labelsize=5)
        a.set_xlabel(x_title, fontsize=7)
        a.set_ylabel(y_title, fontsize=7);
        a.tick_params(rotation=0, labelsize=6)

In [33]:
def add_data(data, new_data, name):
    columns = pd.MultiIndex.from_product([[name], metrics])
    new_df = pd.DataFrame(new_data, index=index, columns=columns)
    concatenate = pd.concat([data, new_df], axis=1)

    return concatenate

In [34]:
def save_report(returns, benchmark="SPY"):
#     color = 'k'
#     plt.rcParams['text.color'] = color
#     plt.rcParams['legend.facecolor'] = 'w'
#     plt.rcParams['xtick.color'] = color
#     plt.rcParams['ytick.color'] = color
#     plt.rcParams['axes.labelcolor'] = color
    
    qs.reports.html(returns, benchmark)
    
#     plt.rcParams['text.color'] = 'w'
#     plt.rcParams['legend.facecolor'] = '#2f3540'
#     plt.rcParams['xtick.color'] = 'w'
#     plt.rcParams['ytick.color'] = 'w'
#     plt.rcParams['axes.labelcolor'] = 'w'

## Performance statistics

In [24]:
def print_backtest_stats_ma(df, fast_ma, slow_ma, ret_strat=np.nan, ir_strat=np.nan, market_ir=np.nan): # Change params fast_ma & slow_ma for long_only
    first_day = df.index[0]
    last_day = df.index[-1]
    loc = full_df.index.get_loc(df.index[0])
    ini_money_market = full_df.iloc[loc - 1]['Close']
    
    ret_market = (df.loc[last_day, 'Close'] / ini_money_market) * 100
    
    print("Period: {:%Y-%m-%d} to {:%Y-%m-%d}".format(df.index[0], df.index[-1]))
    print("\tOverall return of SP500: {:.2f} %. IR of SP500: {:.2f}".format(ret_market, market_ir))
    if (fast_ma >= slow_ma):
        print("\tOverall return of long only: {:.2f} %. IR strategy: {:.2f}. ".format(ret_market, market_ir))
    else:
        print("\tOverall return of {}-{} MA crossover: {:.2f} %. IR strategy: {:.2f}".format(fast_ma, slow_ma, ret_strat, ir_strat))

    return

In [25]:
def calculate_annualized_return_compounded(log_ret):
    arc = 252 * np.mean(log_ret)
    
    return np.round(arc*100, 2)

In [26]:
def calculate_information_ratio(cummulative_ret, ini_equity=ini_equity_default):
    #print("Performance statistics using np.log(Close/Close[-1]): -> PERFECT")
    log_ret = np.log(cummulative_ret / cummulative_ret.shift(1, fill_value=ini_equity))
    
    arc = calculate_annualized_return_compounded(log_ret)
    asd = calculate_annualized_st_dev(log_ret)
    if asd == 0:
        return 0
    
    ir = arc/asd
    
    return np.round(ir, 3)

In [27]:
def calculate_annualized_st_dev(log_ret):
    asd = np.sqrt(252) * log_ret.std()
    
    return np.round(asd*100, 2)

In [76]:
def calculate_performance_metrics(df, ini_equity=ini_equity_default):
    row_name = ['MA_Crossover']
    metrics = ['ARC', 'IR', 'aSD', 'MD', 'AMD', 'MLD', 'All Risk', 'ARCMD', 'ARCAMD']

    # Strategy
    log_ret = np.log(df['Strat_cum_ret'] / df['Strat_cum_ret'].shift(1, fill_value=ini_equity))

    AbsRet = (df['Strat_cum_ret'][-1]/ini_equity - 1) * 100
    ARC = calculate_annualized_return_compounded(log_ret)
    IR = calculate_information_ratio(df['Strat_cum_ret'], ini_equity=ini_equity)
    aSD = calculate_annualized_st_dev(log_ret)
    MD = abs(qs.stats.max_drawdown(df['Strat_cum_ret']))*100
    AMD = abs(df['Strat_cum_ret'].groupby(by=df.index.year).apply(qs.stats.max_drawdown).mean()*100)
    MLD = qs.stats.drawdown_details(qs.stats.to_drawdown_series(df['Strat_cum_ret']))['days'].max()/365.25
    all_risk = aSD*MD*AMD*MLD / 10000
    ARCMD = ARC/MD
    ARCAMD = ARC/AMD

    data = [[ARC, IR, aSD, MD, AMD, MLD, all_risk, ARCMD, ARCAMD]]
    row = pd.DataFrame(data=data, index=row_name, columns=metrics)

    performance_metrics = pd.DataFrame(row).round(3) 
    
    # Buy & Hold
    buy_and_hold = calculate_performance_metrics_buy_and_hold(df, ini_equity=ini_equity)
        
    performance_metrics = pd.concat([performance_metrics, buy_and_hold], axis='index')
    
    return performance_metrics

In [77]:
def calculate_performance_metrics_buy_and_hold(df, ini_equity=ini_equity_default):
    row_name = ['Buy&Hold']
    metrics = ['ARC', 'IR', 'aSD', 'MD', 'AMD', 'MLD', 'All Risk', 'ARCMD', 'ARCAMD']

    log_ret = np.log(df['Market_cum_ret'] / df['Market_cum_ret'].shift(1, fill_value=ini_equity))

    AbsRet = (df['Market_cum_ret'][-1]/ini_equity - 1) * 100
    ARC = calculate_annualized_return_compounded(log_ret)
    IR = calculate_information_ratio(df['Market_cum_ret'], ini_equity=ini_equity)
    aSD = calculate_annualized_st_dev(log_ret)
    MD = abs(qs.stats.max_drawdown(df['Market_cum_ret']))*100
    AMD = abs(df['Market_cum_ret'].groupby(by=df.index.year).apply(qs.stats.max_drawdown).mean()*100)
    MLD = qs.stats.drawdown_details(qs.stats.to_drawdown_series(df['Market_cum_ret']))['days'].max()/365.25
    all_risk = aSD*MD*AMD*MLD / 10000
    ARCMD = ARC/MD
    ARCAMD = ARC/AMD

    data = [[ARC, IR, aSD, MD, AMD, MLD, all_risk, ARCMD, ARCAMD]]
    row = pd.DataFrame(data=data, index=row_name, columns=metrics)

    performance_metrics = pd.DataFrame(row).round(3) 
        
    return performance_metrics